# Extend The 44 vertical levels to 46

In [1]:
import os
import numpy as np
import xarray as xr
from struct import *
from MITgcmutils import mds

## I. Functions

## II. Set paths

In [2]:
# Set parameters
HDlist = os.listdir('/Volumes') 
HDsel = np.nan
HD = ["Proteus","Itsasorantz"]
# HD Selection
while not (HDsel==1 or HDsel==2):
    # While loop Intilization
    if not np.isnan(HDsel):
        print("Wrong choice ! Type 1 or 2")
    # Test Both HD pluged in case    
    if HD[0] in HDlist and HD[1] in HDlist:
        HDsel = int(input("Do you want to read data on (1) Proteus or (2) Itsasorantz ?"))
    # Selection for 1 HD pluged in
    elif HD[0] in HDlist:
        HDsel = 1
    elif HD[1] in HDlist:
        HDsel = 2
print(f"Reading data on {HD[HDsel-1]}")

Reading data on Itsasorantz


In [3]:
HDpath = f"/Volumes/{HD[HDsel-1]}/"
In_pth = HDpath+"Modelling/Forcing/run_template/Mac270_Carroll/32_Tracers_Nr44/"
Out_pth = HDpath+"Modelling/Forcing/run_template/Mac270_Carroll/32_Tracers_Nr46/"
grid_pth = HDpath+"Modelling/Forcing/grids/mac270/G46x68x44/"

In [4]:
itr = 420768

## III. Add 2 top layers

### a. Read files

In [5]:
darwin, _, darwin_meta = mds.rdmds(In_pth+'pickup_darwin', itrs=itr, returnmeta=True)
ggl90, _, ggl90_meta = mds.rdmds(In_pth+'pickup_ggl90', itrs=itr, returnmeta=True)
ptracers, _, ptracers_meta = mds.rdmds(In_pth+'pickup_ptracers', itrs=itr, returnmeta=True)
pickup, _, pickup_meta = mds.rdmds(In_pth+'pickup', itrs=itr, returnmeta=True)
# >> Note for following files:
#    copy pickup_darwin meta file and rename with the name of the file
#smooth3D, _, smooth3D_meta = mds.rdmds(In_pth+'smooth3DscalesH001', itrs=1, returnmeta=True)
##    for following files change dataprec = float64 by float32
#smooth, _, smooth_meta = mds.rdmds(In_pth+'smooth3Dnorm001', itrs=1, returnmeta=True)
#xx_diffkr, _, xx_diffkr_meta = mds.rdmds(In_pth+'xx_diffkr', itrs=42, returnmeta=True)
#xx_kapgm, _, xx_kapgm_meta = mds.rdmds(In_pth+'xx_kapgm', itrs=42, returnmeta=True)
#xx_kapredi, _, xx_kapredi_meta = mds.rdmds(In_pth+'xx_kapredi', itrs=42, returnmeta=True)
#wdiffkrFld, _, wdiffkrFld_meta = mds.rdmds(In_pth+'wdiffkrFld_46x68_Mac', itrs=1, returnmeta=True)
#wkapgmFld, _, wkapgmFld_meta = mds.rdmds(In_pth+'wkapgmFld_46x68_Mac', itrs=1, returnmeta=True)
#wkaprediFld, _, wkaprediFld_meta = mds.rdmds(In_pth+'wkaprediFld_46x68_Mac', itrs=1, returnmeta=True)

### b. Duplicate top layer 2 times

In [6]:
# Create parameter to repeat first layer
r = np.ones(44)
r[0] = 3

In [7]:
## Add layer
darwin_ext = np.repeat(darwin, list(r), axis=0)
ggl90_ext = np.repeat(ggl90, list(r), axis=0)
ptracers_ext = np.repeat(ptracers, list(r), axis=1)
#smooth_ext = np.repeat(smooth, list(r), axis=0)
#smooth3D_ext = np.repeat(smooth3D, list(r), axis=0)
#xx_diffkr_ext = np.repeat(xx_diffkr, list(r), axis=0)
#xx_kapgm_ext = np.repeat(xx_kapgm, list(r), axis=0)
#xx_kapredi_ext = np.repeat(xx_kapredi, list(r), axis=0)
#wdiffkrFld_ext = np.repeat(wdiffkrFld, list(r), axis=0)
#wkapgmFld_ext = np.repeat(wkapgmFld, list(r), axis=0)
#wkaprediFld_ext = np.repeat(wkaprediFld, list(r), axis=0)

In [8]:
## Add layer specifically for pickup
pickup_ls = []
for i in range(8):
    pickup_ls.append(np.repeat(pickup[44*i:44*(i+1)], list(r), axis=0))
pickup_ext = np.concatenate(pickup_ls, axis=0)
pickup_ext = np.concatenate((pickup_ext,pickup[352:]),axis=0)

### c. Create additional smooth files

In [9]:
## Array sizes
nZ = 46
nX = 68
nY = 46
## cte
fac=3
facZ=2

In [10]:
#dzi = mds.rdmds(grid_pth+'DRC')
## Modify dz
#dz0 = np.ones((4,1,1))
#dz0[0] = 1.5; dz0[1] = 3
#dz0[2] = 3.5; dz0[3] = 7
#dz = np.concatenate((dz0,dzi[2:-1]))
#dz = np.repeat(np.repeat(dz, nX, axis=1),nY,axis=2)
## Create smooth3DscalesZ001 matrix
#dz1 = dz*facZ

### d. Save files

In [11]:
def bin_save(fnm, array):
    newFile = open(fnm, "wb")
    bitearray = pack(">%sf" % len(array.flatten()),*array.flatten())
    newFile.write(bitearray);

In [12]:
svc = input("Do you want to save the New Pickup file created ? ('y' or 'n')\n")
if svc == 'y':
    # darwin
    mds.wrmds(Out_pth+'pickup_darwin', darwin_ext, itr=itr, dataprec='float64', ndims=darwin_meta['ndims'][0],
              nrecords=darwin_meta['nrecords'][0], fields=darwin_meta['fldlist'], dimlist=[46, 68, 46])
    # ggl90
    mds.wrmds(Out_pth+'pickup_ggl90', ggl90_ext, itr=itr, dataprec='float64', ndims=ggl90_meta['ndims'][0], 
              nrecords=ggl90_meta['nrecords'][0], dimlist=[46, 68, 46])
    # ptracers
    mds.wrmds(Out_pth+'pickup_ptracers', ptracers_ext, itr=itr, dataprec='float64', ndims=ptracers_meta['ndims'][0],
              nrecords=ptracers_meta['nrecords'][0], fields=ptracers_meta['fldlist'], deltat=1200,
              dimlist=[46, 68, 46])
    # pickup
    mds.wrmds(Out_pth+'pickup', pickup_ext, itr=itr, dataprec='float64', ndims=pickup_meta['ndims'][0],
              nrecords=371, fields=pickup_meta['fldlist'], dimlist=[46, 68])
#    # smooth
#    mds.wrmds(Out_pth+'smooth3Dnorm001', smooth_ext, itr=smooth_meta['timestepnumber'][0], dataprec='float32', 
#              ndims=smooth_meta['ndims'][0], nrecords=1,dimlist=[46, 68, 46])
#    # smooth3DH
#    mds.wrmds(Out_pth+'smooth3DscalesH001', smooth3D_ext, itr=smooth3D_meta['timestepnumber'][0], 
#              dataprec='float64', ndims=smooth3D_meta['ndims'][0], nrecords=1,dimlist=[46, 68, 46])
#    # smooth3DZ
#    bin_save(Out_pth+'smooth3DscalesZ001.data', dz1)
#    # xx_diffkr
#    mds.wrmds(Out_pth+'xx_diffkr', xx_diffkr_ext, itr=42, dataprec='float32', ndims=xx_diffkr_meta['ndims'][0],
#              nrecords=xx_diffkr_meta['nrecords'][0], dimlist=[46, 68, 46])
#    # xx_kapgm
#    mds.wrmds(Out_pth+'xx_kapgm', xx_kapgm_ext, itr=42, dataprec='float32', ndims=xx_kapgm_meta['ndims'][0],
#              nrecords=xx_kapgm_meta['nrecords'][0], dimlist=[46, 68, 46])
#    # xx_kapredi
#    mds.wrmds(Out_pth+'xx_kapredi', xx_kapredi_ext, itr=42, dataprec='float32', ndims=xx_kapredi_meta['ndims'][0],
#              nrecords=xx_kapredi_meta['nrecords'][0], dimlist=[46, 68, 46])
#    # wdiffkrFld
#    mds.wrmds(Out_pth+'wdiffkrFld_46x68_Mac', wdiffkrFld_ext, itr=1, dataprec='float32', ndims=wdiffkrFld_meta['ndims'][0],
#              nrecords=wdiffkrFld_meta['nrecords'][0], dimlist=[46, 68, 46])
#    # wkapgmFld
#    mds.wrmds(Out_pth+'wkapgmFld_46x68_Mac', wkapgmFld_ext, itr=1, dataprec='float32', ndims=wkapgmFld_meta['ndims'][0],
#              nrecords=wkapgmFld_meta['nrecords'][0], dimlist=[46, 68, 46])
#    # wkaprediFld
#    mds.wrmds(Out_pth+'wkaprediFld_46x68_Mac', wkaprediFld_ext, itr=1, dataprec='float32', ndims=wkaprediFld_meta['ndims'][0],
#              nrecords=wkaprediFld_meta['nrecords'][0], dimlist=[46, 68, 46])
    
    print("proceed")
elif svc == 'n':
    print("Warning: files not saved")

Do you want to save the New Pickup file created ? ('y' or 'n')
y
proceed
